## Notebook Outline

- [Import libraries](#Import-libraries)
- [Import data](#Import-data)
- [Identify player name](#Identify-player-name)
- [Add player ids to 'slg_stats_2010.csv'](#Add-player-ids-to-'slg_stats_2010.csv')
- [Pitch Types](#Pitch-Types)
- [Feature Engineering: Last Play](#Feature-Engineering:-Last-Play)
- [Feature Engineering: Last 10 Pitches](#Feature-Engineering:-Last-10-Pitches)
- [Appendix](#Appendix)

---

#### Import libraries

In [1]:
%%capture
cd ..

In [2]:
from __future__ import division
from functions.encoders.OneHotLabelEncoder import OneHotLabelEncoder

import numpy as np
import pandas as pd
import pandas_profiling

pd.set_option("display.max_columns", 200)

---

#### Import data

In [3]:
# pitch data
df = pd.read_csv("data/pitches.csv", parse_dates=['date'])

# metadata
df_meta = pd.read_csv("data/pitch_by_pitch_metadata.csv")

# position data scraped from baseball-reference.com
df_pos = pd.read_csv("data/positions.csv")
df_pos = df_pos.set_index('player')

# slg data scraped from baseball-reference.com
df_slg = pd.read_csv("data/slg_stats_2010.csv")
df_slg = df_slg.set_index('player')

/Users/athai/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


[See Appendix for preview of df](#Appendix)

- 'pitches.csv' contatins a lot of columns with NaN columns
- to make the dataset more manageable to work with, let's filter the data to columns that are available prior to the pitch

In [5]:
# metadata
df_meta = pd.read_csv("data/pitch_by_pitch_metadata.csv")

avail_cols = df_meta['column_name'][df_meta['available_prior_to_pitch'] == "Yes"]

# add target variable
avail_cols = ['pitch_type'] + avail_cols.tolist()

[See Appendix for output of avail_cols](#Appendix)

- we are now working with 29 columns of data
- columns that describe the result of the play (final_balls, final_strikes, event, etc.) are not included
- PITCHf/x columns are not included

In [7]:
# describe column data in dataframe with pandas_profiling
pfr = pandas_profiling.ProfileReport(df.loc[:, avail_cols])

[See Appendix for output of pfr](#pfr)  
Alternatively, open "available_prior_to_pitch.html" to view results in new window

**Observations of interest**

- Transformations
 - **b_height** has string data, will need to convert to numeric
 - **on_1b**, **on_2b**, **on_3b** contain mostly missing values. When not missing, value indicates the batter_id of person on base. Should create binary features that indicates whether a batter is on base or not  


- Potential columns to drop
 - **at_bat_num**, **pitch_id** is correlated with **inning**
 - **Year** is always 2011


- identification columns
 - **batter_id**
 - **game_pk**
 - **pitcher_id**
 - **team_id_b**
 - **team_id_p**
 - **uid**


- date/time columns
 - **date**
 - **start_tfs**
 - **start_tfs_zulu**
 
 
- binary coulmns
 - **p_throws**: L and R
 - **stand**: L and R
 - **top**: 1 and 0
 
 
- target column
 - **pitch type** is 0.3% missing
 - 19 unique values, mostly fastball pitches
 
 
Overall dataset is very clean, only 0.3% of data (missing pitch_type) is unusable

In [9]:
# filter data to columns available prior to pitch (and 'at_bat_des' for batter id identification)
# and drop rows with missing pitch_type

print 'dimensions before dropping data:', df.shape
df = df.loc[pd.notnull(df['pitch_type']), avail_cols + ['at_bat_des']]
print 'dimensions after dropping data:', df.shape

dimensions before dropping data: (718961, 125)
dimensions after dropping data: (716681, 30)


---

#### Identify player name

The name of the player is usually listed at the beginning of the text found in the 'at_bat_des' column

In [10]:
df['batter'] = df['at_bat_des']

# remove spaces from players with abbreviated first names
df['batter'] = df['batter'].str.replace('^(([A-Z]\.)\s\s([A-Z]\.)\s\s)', '\\2\\3')
df['batter'] = df['batter'].str.replace('^(J\.   Hardy)', 'J.J. Hardy')

# extract batter name
df['batter'] = df['batter'].str.extract('([^\s]*\s[^\s]*)(?=\s)', expand=False)

However, when a pitcher intentionally walks a batter, the pitcher's name is found first in 'at_bat_des'  
We can correct this by returning the most frequent batter name per player id

In [11]:
# find most frequent 'batter' label based on 'at_bat_des' data
g = df.groupby(['batter_id'])
g = g['batter'].agg(lambda x: x.value_counts().index[0])

# fix bad names from regex extract (names not found in scraped 'positions.csv' file)
name_recode = {'Wily Mo': 'Wily Mo Pena',
               'Jorge De': 'Jorge De La Rosa',
               'Juan Carlos': 'Juan Carlos Oviedo',
               'Alejandro De': 'Alejandro De Aza',
               'Ivan De': 'Ivan De Jesus',
               'Rubby De': 'Rubby De La Rosa'}

# update strings for bad names
for i, x in g.iteritems():
    if x in name_recode:
        g.loc[i] = name_recode[x]

# delete old batter column and join with corrected names
del df['batter']
df = df.join(g, on='batter_id')

[See Appendix for regex verification](#regex)

---

#### Add player ids to 'slg_stats_2010.csv'

In [15]:
# Average 2010 SLG data by position, used when AB is too small to be reliable
# Found at https://www.baseball-reference.com/leagues/team_compare.cgi?request=1&year=2010&lg=MLB&stat=SLG

slgs = {'avg': .403,
        'P': .174,
        'C': .381,
        '1B': .452,
        '2B': .389,
        '3B': .418,
        'SS': .374,
        'LF': .432,
        'CF': .405,
        'RF': .442,
        'DH': .422}

In [16]:
slg_table = []

for i, x in g.iteritems():
    try:
        # idenitified a position for the batter in 'positions.csv'
        pos = df_pos.loc[x].position
        
        try:
            # identified 2010 SLG data for the batter
            # batter also has >= 100 AB due to how 'slg_stats_2010.csv' was filtered
            slg = df_slg.loc[x].slg
            slg_table.append([i, x, pos, slg])
        except:
            # insufficient AB
            # use average SLG for batter's position in 2010
            slg_table.append([i, x, pos, slgs.get(pos, .403)])
    except:
        # no position found
        # use average overall SLG in 2010
        slg_table.append([i, x, '', .403])

# turn table in DataFrame
slg_table = pd.DataFrame(slg_table, columns=['id', 'batter', 'position', 'slg_2010'])
slg_table = slg_table.set_index('id')

# fix data for the two Chris Youngs (NYM - team_id: 121, ARI - team_id: 109)
slg_table.loc[432934, 'position'] = 'P'
slg_table.loc[432934, 'slg_2010'] = slgs['P']
slg_table.loc[455759, 'position'] = 'CF'
slg_table.loc[455759, 'slg_2010'] = 0.452

[See Appendix for slg_table verification](#slg_table)

---

#### Pitch Types

[This website](https://www.fangraphs.com/library/pitch-type-abbreviations-classifications/) provides descriptions of the pitch type codes. Most type of pitches are either Fastballs or Off-speeds.  
Additionally, [wikipedia](http://www.wikiwand.com/en/Screwball#/References) also provides Fastball / Off-speed / Purpose Pitch classifications for pitch_type.

I decided to recode the pitches based on the pitch types specified in an ESPN baseball game log [play-by-play log](http://www.espn.com/mlb/playbyplay?gameId=371101119#allPlays3711011190001).

In [20]:
def recode_pitch_type(val):        
    pitch_dict = {'SI': 'Sinker', 'SL': 'Slider', 'CU': 'Curveball', 
                  'KC': 'Curveball', 'CH': 'Changeup', 'FC': 'Cutter',
                  'UN': 'Unidentified'}
    
    fastballs = set(['FF', 'FT', 'FS', 'FA'])
    purpose_pitches = set(['IN', 'PO', 'FO', 'AB'])
    off_speed = set(['KN', 'EP', 'SC'])
    
    pitch_dict.update({x: 'Fastball' for x in fastballs})
    pitch_dict.update({x: 'Purpose_Pitch' for x in purpose_pitches})
    pitch_dict.update({x: 'Off-Speed' for x in off_speed})
    
    return pitch_dict[val]

In [21]:
df.loc[:, 'pitch_type'] = df.loc[:, 'pitch_type'].map(lambda x: recode_pitch_type(x))
df['pitch_type'].value_counts()/len(df)

Fastball         0.460880
Slider           0.153145
Sinker           0.122425
Changeup         0.101358
Curveball        0.090513
Cutter           0.058188
Purpose_Pitch    0.006904
Off-Speed        0.006564
Unidentified     0.000024
Name: pitch_type, dtype: float64

In [22]:
# Dropping Unidentified from dataset due to extreme minority (0.002%)

mask = df['pitch_type'] == 'Unidentified'

print 'dimensions before dropping data:', df.shape
df = df.loc[~mask, :]
print 'dimensions after dropping data:', df.shape

dimensions before dropping data: (716681, 31)
dimensions after dropping data: (716664, 31)


---

#### Feature Engineering: Last Play

Feature engineering for data that comes as a direct result of the last play (grouped by at_bat)

- **last_pitch**
 - lag data for last pitch
 - if first_at_bat, return N/A
 - perhaps pitchers are more or less likely to throw the same pitch type consectutively?


- **last_count_type**
 - was the last pitch a foul, ball, or strike?
 - if first at_bat, return N/A
 - perhaps pitchers are more or less likely to pitch differently based on the outcome of the last play?

In [23]:
def last_data(df):
    subset_cols = ['pcount_at_bat', 'balls', 'strikes', 'fouls', 'pitch_type']
    df_temp = df.loc[:, subset_cols]
    
    df_temp.loc[:, 'last_balls'] = df_temp['balls'].shift(1)
    df_temp.loc[:, 'last_strikes'] = df_temp['strikes'].shift(1)
    df_temp.loc[:, 'last_fouls'] = df_temp['fouls'].shift(1)
    df_temp.loc[:, 'last_pitch'] = df_temp['pitch_type'].shift(1)
    
    def last_count_type(row):
        if row['pcount_at_bat'] == 1:
            return 'N/A'
        elif row['fouls'] - row['last_fouls']:
            return 'Foul'
        elif row['balls'] - row['last_balls']:
            return 'Ball'
        else:
            return 'Strike'
        
    def last_pitch(row):
        if row['pcount_at_bat'] == 1:
            return 'N/A'
        else:
            return row['last_pitch']
    
    df_temp.loc[:, 'last_count_type'] = df_temp.apply(lambda x: last_count_type(x), axis=1)
    df_temp.loc[:, 'last_pitch'] = df_temp.apply(lambda x: last_pitch(x), axis=1)
    
    return pd.concat([df, df_temp[['last_count_type', 'last_pitch']]], axis=1)

In [24]:
df = last_data(df)

[See Appendix for last play verification](#last_play)

---

#### Feature Engineering: Last 10 Pitches

Feature engineering for the pitcher's last 10 pitches (within the same game)

- **{pitch_type}_L10**
 - count of {pitch_type} thrown by pitcher in last 10 plays within the same game
 - if first pitch by pitcher in game, return -1
 - if less than 10 previous plays are available, sum of all previous plays are returned

In [28]:
def addL10(df):
    df = OneHotLabelEncoder(['pitch_type'], delete=False).fit_transform(df)
    
    rolling_cols = ['pitch_type_Changeup', 'pitch_type_Curveball', 'pitch_type_Cutter',
                    'pitch_type_Fastball', 'pitch_type_Off-Speed', 'pitch_type_Purpose_Pitch',
                    'pitch_type_Sinker', 'pitch_type_Slider']

    for col in rolling_cols:
        # shift the pitch_type data down one row so current pitch_type is not included in rolling sum
        df[col + '_shifted'] = df.groupby(['game_pk', 'pitcher_id'])[col].shift(1)

        # calculate rolling sum of each pitch type for last 10 plays by pitcher
        # returns -1 if this is the pitcher's first pitch in the game
        # if less than 10 previous plays are available, sum of all previous plays are returned
        df[col[11:] + '_L10'] = df.groupby(['game_pk', 'pitcher_id'])[col + '_shifted']\
                            .apply(lambda x: x.rolling(center=False, window=10, min_periods=1)\
                            .sum()).fillna(-1)
        
        # delete intermediate columns
        del df[col + '_shifted']
        del df[col]
        
    return df

In [29]:
df = addL10(df)

[See Appendix for last 10 pitches verification](#last10_pitches)

#### Train-Dev-Test Split

Split data into train, dev, and test datasets  
Since the data includes time data, it does not make sense to random split the data  
Instead, let's split the data by the month of year

In [34]:
df['month'] = df['date'].map(lambda x: x.month)
df['month'].value_counts(sort=False)

3       1778
4     114347
5     122142
6     114958
7     113969
8     121886
9     117090
10     10494
Name: month, dtype: int64

Assign datasets as follows:  
  
- Train (for training models): data up to July 2011  
- Dev (for validating training models): August 2011    
- Test (for evaluating machine learning models): September and October 2011

In [35]:
# retain original 29 df_meta columns
train = df.loc[df.month <= 7].iloc[:, 0:29]
dev = df.loc[df.month == 8].iloc[:, 0:29]
test = df.loc[df.month >= 9].iloc[:, 0:29]

#### Output data

In [36]:
# create data frame consisting of uid (row identifier), last_pitch, last_count, and L10 features
df_engineered_pitch = df.iloc[:, np.r_[1, -11:-1]]
df_engineered_pitch.to_csv('data/engineered_pitch_data.csv', index=False)

train.to_csv('data/train.csv', index=False)
dev.to_csv('data/dev.csv', index=False)
test.to_csv('data/test.csv', index=False)

slg_table.to_csv('data/slg_2010_final.csv')

---

### Appendix

In [4]:
df.head()

,uid,game_pk,year,date,team_id_b,team_id_p,inning,top,at_bat_num,pcount_at_bat,pcount_pitcher,balls,strikes,fouls,outs,is_final_pitch,final_balls,final_strikes,final_outs,start_tfs,start_tfs_zulu,batter_id,stand,b_height,pitcher_id,p_throws,at_bat_des,event,event2,event3,event4,away_team_runs,home_team_runs,score,pitch_des,pitch_id,type,pitch_tfs,pitch_tfs_zulu,x,y,sv_id,start_speed,end_speed,sz_top,sz_bot,pfx_x,pfx_z,px,pz,x0,z0,y0,vx0,vz0,vy0,ax,az,ay,break_length,break_y,break_angle,pitch_type,type_confidence,zone,nasty,spin_dir,spin_rate,cc,on_1b,on_2b,on_3b,runner1_id,runner1_start,runner1_end,runner1_event,runner1_score,runner1_rbi,runner1_earned,runner2_id,runner2_start,runner2_end,runner2_event,runner2_score,runner2_rbi,runner2_earned,runner3_id,runner3_start,runner3_end,runner3_event,runner3_score,runner3_rbi,runner3_earned,runner4_id,runner4_start,runner4_end,runner4_event,runner4_score,runner4_rbi,runner4_earned,runner5_id,runner5_start,runner5_end,runner5_event,runner5_score,runner5_rbi,runner5_earned,runner6_id,runner6_start,runner6_end,runner6_event,runner6_score,runner6_rbi,runner6_earned,runner7_id,runner7_start,runner7_end,runner7_event,runner7_score,runner7_rbi,runner7_earned,created_at,added_at,modified_at,modified_by
0,14143226,286874,2011,2011-03-31,108,118,1,1,1,1,1,0,0,0,0,0,2,1,1,201226,2011-03-31 20:12:26,430895,L,5-8,460024,R,"Maicer Izturis grounds out, second baseman Chr...",Groundout,NaN,NaN,NaN,0,0,NaN,Ball,3,B,201301.0,2011-03-31 20:13:01,105.58,180.46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-03-03 21:33:20,2016-03-03 21:33:20,2016-03-03 21:33:20,1
1,14143227,286874,2011,2011-03-31,108,118,1,1,1,2,2,1,0,0,0,0,2,1,1,201226,2011-03-31 20:12:26,430895,L,5-8,460024,R,"Maicer Izturis grounds out, second baseman Chr...",Groundout,NaN,NaN,NaN,0,0,NaN,Ball,4,B,201319.0,2011-03-31 20:13:19,99.57,170.96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-03-03 21:33:20,2016-03-03 21:33:20,2016-03-03 21:33:20,1
2,14143228,286874,2011,2011-03-31,108,118,1,1,1,3,3,2,0,0,0,0,2,1,1,201226,2011-03-31 20:12:26,430895,L,5-8,460024,R,"Maicer Izturis grounds out, second baseman Chr...",Groundout,NaN,NaN,NaN,0,0,NaN,Called Strike,5,S,201327.0,2011-03-31 20:13:27,95.28,152.83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-03-03 21:33:20,2016-03-03 21:33:20,2016-03-03 21:33:20,1
3,14143229,286874,2011,2011-03-31,108,118,1,1,1,4,4,2,1,0,0,1,2,1,1,201226,2011-03-31 20:12:26,430895,L,5-8,460024,R,"Maicer Izturis grounds out, second baseman Chr...",Groundout,NaN,NaN,NaN,0,0,NaN,"In play, out(s)",6,X,180441.0,2011-03-31 18:04:41,93.56,168.37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-03-03 21:33:20,2016-03-03 21:33:20,2016-03-03 21:33:20,1
4,14143230,286874,2011,2011-03-31,108,118,1,1,2,1,5,0,0,0,1,0,2,2,1,201354,2011-03-31 20:13:54,435062,R,5-10,460024,R,Howie Kendrick doubles (1) on a line drive to ...,Double,NaN,NaN,NaN,0,0,N

In [6]:
avail_cols

['pitch_type',
 'uid',
 'game_pk',
 'year',
 'date',
 'team_id_b',
 'team_id_p',
 'inning',
 'top',
 'at_bat_num',
 'pcount_at_bat',
 'pcount_pitcher',
 'balls',
 'strikes',
 'fouls',
 'outs',
 'start_tfs',
 'start_tfs_zulu',
 'batter_id',
 'stand',
 'b_height',
 'pitcher_id',
 'p_throws',
 'away_team_runs',
 'home_team_runs',
 'pitch_id',
 'on_1b',
 'on_2b',
 'on_3b']

[Return to notebook](#Import-data)
<a id="pfr"></a>

In [8]:
pfr.to_file("notebooks/html/available_prior_to_pitch.html")
pfr

Number of variables,29
Number of observations,718961
Total Missing (%),8.3%
Total size in memory,159.1 MiB
Average record size in memory,232.0 B
Numeric,20
Categorical,5
Date,1
Text (Unique),0
Rejected,3
Correlation,0.97635


[Return to notebook](#Import-data)
<a id="regex"></a>

---
Check regex

In [12]:
print 'pre-regex:'
print df.loc[26, 'at_bat_des']
print df.loc[9288, 'at_bat_des']
print df.loc[2486, 'at_bat_des']
print df.loc[318066, 'at_bat_des']

test_df = pd.Series([df.loc[26, 'at_bat_des'], df.loc[9288, 'at_bat_des'],
                     df.loc[2486, 'at_bat_des'], df.loc[318066, 'at_bat_des']])

print
print 'post-regex:'
print test_df.str.extract('([^\s]*\s[^\s]*)(?=\s)', expand=False).loc[0]
print test_df.str.replace('^(([A-Z]\.)\s\s([A-Z]\.)\s\s)', '\\2\\3').loc[1]
print test_df.str.replace('^(J\.   Hardy)', 'J.J. Hardy').loc[2]
print test_df.str.extract('([^\s]*\s[^\s]*)(?=\s)', expand=False).loc[3]

pre-regex:
Mike Aviles called out on strikes.  
A.  J.   Ellis singles on a ground ball to second baseman Freddy Sanchez.  
J.   Hardy doubles (1) on a fly ball to center fielder B.   Upton.  
Wily Mo Pena pops out to first baseman Eric Hosmer in foul territory.  

post-regex:
Mike Aviles
A.J. Ellis singles on a ground ball to second baseman Freddy Sanchez.  
J.J. Hardy doubles (1) on a fly ball to center fielder B.   Upton.  
Wily Mo


In [13]:
print df.loc[266805, 'at_bat_des']

A.  J.   Burnett intentionally walks J.  D.   Drew.  


In [14]:
df[['batter_id', 'batter']].loc[[26, 9288, 2486, 266805, 318066]]

,batter_id,batter
26,449107,Mike Aviles
9288,454560,A.J. Ellis
2486,429666,J.J. Hardy
266805,136770,J.D. Drew
318066,276377,Wily Mo Pena


[Return to notebook](#Identify-player-name)
<a id="slg_table"></a>

---
Check if Chris Young data in **slg_table** is correct

In [17]:
df.loc[[39200, 711938], ['batter', 'batter_id', 'top', 'team_id_b']]

,batter,batter_id,top,team_id_b
39200,Chris Young,432934,0,121
711938,Chris Young,455759,0,109


In [18]:
slg_table[slg_table.batter == 'Chris Young']

,batter,position,slg_2010
id,,,
432934,Chris Young,P,0.174
455759,Chris Young,CF,0.452


In [19]:
slg_table.head()

,batter,position,slg_2010
id,,,
110029,Bobby Abreu,DH,0.435
110683,Miguel Batista,P,0.174
111072,Henry Blanco,C,0.300
111851,Orlando Cabrera,SS,0.354
111867,Miguel Cairo,3B,0.410


[Return to notebook](#Add-player-ids-to-'slg_stats_2010.csv')
<a id="last_play"></a>

---

Check data for **last_pitch** and **last_count_type** columns

In [25]:
df[['pitch_type', 'last_pitch', 'pcount_at_bat', 'balls', 'strikes', 'fouls', 'last_count_type']].head(15)

,pitch_type,last_pitch,pcount_at_bat,balls,strikes,fouls,last_count_type
26,Fastball,N/A,1,0,0,0,N/A
27,Fastball,Fastball,2,0,1,0,Strike
28,Fastball,Fastball,3,1,1,0,Ball
29,Fastball,Fastball,4,2,1,0,Ball
30,Fastball,Fastball,5,2,2,1,Foul
31,Slider,Fastball,6,2,2,2,Foul
32,Fastball,Slider,7,3,2,2,Ball
33,Fastball,Fastball,8,3,2,3,Foul
34,Slider,Fastball,9,3,2,4,Foul
35,Fastball,N/A,1,0,0,0,N/A


[Return to notebook](#Feature-Engineering:-Last-Play)
<a id="last10_pitches"></a>

---

Check data in **L10 columns** in game_pk == 287372 for:
- pitcher_id 430912 (starting pitcher, road team)
- pitcher_id 434622 (starting pitcher, home team)
- pitcher_id 434637 (7th inning pitcher, home team)

In [30]:
cols = ['pitch_id', 'pitch_type', 'Changeup_L10', 'Curveball_L10', 'Cutter_L10',
        'Fastball_L10', 'Off-Speed_L10', 'Purpose_Pitch_L10', 'Sinker_L10', 'Slider_L10']

In [31]:
df.loc[np.logical_and(df['game_pk'] == 287372, df['pitcher_id'] == 430912), cols].head(20)

,pitch_id,pitch_type,Changeup_L10,Curveball_L10,Cutter_L10,Fastball_L10,Off-Speed_L10,Purpose_Pitch_L10,Sinker_L10,Slider_L10
137416,3,Fastball,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
137417,4,Fastball,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
137418,5,Changeup,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
137419,6,Changeup,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
137420,10,Fastball,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
137421,11,Fastball,2.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
137422,12,Fastball,2.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
137423,13,Fastball,2.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
137424,14,Fastball,2.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0
137425,15,Fastball,2.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0


In [32]:
df.loc[np.logical_and(df['game_pk'] == 287372, df['pitcher_id'] == 434622), cols].head(20)

,pitch_id,pitch_type,Changeup_L10,Curveball_L10,Cutter_L10,Fastball_L10,Off-Speed_L10,Purpose_Pitch_L10,Sinker_L10,Slider_L10
137428,25,Fastball,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
137429,26,Fastball,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
137430,27,Fastball,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
137431,28,Curveball,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
137432,29,Slider,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0
137433,30,Fastball,0.0,1.0,0.0,3.0,0.0,0.0,0.0,1.0
137434,34,Curveball,0.0,1.0,0.0,4.0,0.0,0.0,0.0,1.0
137435,35,Fastball,0.0,2.0,0.0,4.0,0.0,0.0,0.0,1.0
137436,36,Slider,0.0,2.0,0.0,5.0,0.0,0.0,0.0,1.0
137437,37,Fastball,0.0,2.0,0.0,5.0,0.0,0.0,0.0,2.0


In [33]:
df.loc[np.logical_and(df['game_pk'] == 287372, df['pitcher_id'] == 434637), cols].head(20)

,pitch_id,pitch_type,Changeup_L10,Curveball_L10,Cutter_L10,Fastball_L10,Off-Speed_L10,Purpose_Pitch_L10,Sinker_L10,Slider_L10
137634,430,Fastball,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
137635,431,Sinker,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
137636,432,Fastball,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
137637,433,Fastball,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0
137638,434,Slider,0.0,0.0,0.0,3.0,0.0,0.0,1.0,0.0
137639,435,Fastball,0.0,0.0,0.0,3.0,0.0,0.0,1.0,1.0
137640,441,Sinker,0.0,0.0,0.0,4.0,0.0,0.0,1.0,1.0
137641,442,Fastball,0.0,0.0,0.0,4.0,0.0,0.0,2.0,1.0
137642,443,Sinker,0.0,0.0,0.0,5.0,0.0,0.0,2.0,1.0
137643,444,Fastball,0.0,0.0,0.0,5.0,0.0,0.0,3.0,1.0


[Return to notebook](#Feature-Engineering:-Last-10-Pitches)
<a id="pitch_cdf"></a>